### Pydatic output parser tutorial

In [1]:
# ----- Part 01
from pydantic import BaseModel, Field
# Define model
# Expect response model   
class FlowerDescription(BaseModel):
    flower_type: str = Field(description="flower type")
    price: int = Field(description="flower price")
    description: str = Field(description="description")
    reason: str = Field(description="Why recommend")
    
# Prepare data
flowers = ["玫瑰", "百合", "康乃馨"]
prices = ["99", "33", "77"]



In [2]:
# ----- Part 02
# Create output parser
from langchain.output_parsers import PydanticOutputParser
output_parser = PydanticOutputParser(pydantic_object=FlowerDescription)

# Get output format instructions
format_instructions = output_parser.get_format_instructions()
print("Output Format:", format_instructions)


Output Format: The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"flower_type": {"description": "flower type", "title": "Flower Type", "type": "string"}, "price": {"description": "flower price", "title": "Price", "type": "integer"}, "description": {"description": "description", "title": "Description", "type": "string"}, "reason": {"description": "Why recommend", "title": "Reason", "type": "string"}}, "required": ["flower_type", "price", "description", "reason"]}
```


In [3]:
# ----- Part 03
# Create prompt
from langchain import PromptTemplate
prompt_template = """
您是一位专业的鲜花店文案撰写员。
对于售价为 {price} 元的 {flower} ，您能提供一个吸引人的简短中文描述吗？
{format_instructions}
"""

# 根据模板提示词，同时在提示词中加入输出解析器的说明
prompt = PromptTemplate.from_template(prompt_template,
                                      partial_variables={"format_instructions": format_instructions}
                                     )
print(prompt)

input_variables=['flower', 'price'] input_types={} partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"flower_type": {"description": "flower type", "title": "Flower Type", "type": "string"}, "price": {"description": "flower price", "title": "Price", "type": "integer"}, "description": {"description": "description", "title": "Description", "type": "string"}, "reason": {"description": "Why recommend", "title": "Reason", "type": "string"}}, "required": ["flower_type", "price", "description", "reason"]}\n```'} template='\n您是一位专业的鲜花店文

In [ ]:
# ---- Part 04
# Test Genereate prompt list
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY")
from langchain_google_genai import ChatGoogleGenerativeAI

chat_model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=1024,
    timeout=None,
    max_retries=2,
)

# 创建一个空的DataFrame用于存储结果
import pandas as pd
df = pd.DataFrame(columns=["flower_type", "price", "description", "reason"])

for flower, price in zip(flowers, prices):
    # 根据提示准备模型的输入
    input = prompt.format(flower=flower, price=price)
    # 打印提示词
    print("Prompt:", input)
    # 调用模型
    output = chat_model.invoke(input)
    # 解析模型的输出
    parsed_output = output_parser.parse(output.content)
    
    print("Parsed_output:", parsed_outup.dict())
    
    # 将解析后的输出添加到DataFrame中
    df.loc[len(df)] = parsed_output.dict()
    
# Print dict
print("Output data", df.to_dict(orient='records'))
    
    

Prompt: 
您是一位专业的鲜花店文案撰写员。
对于售价为 99 元的 玫瑰 ，您能提供一个吸引人的简短中文描述吗？
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"flower_type": {"description": "flower type", "title": "Flower Type", "type": "string"}, "price": {"description": "flower price", "title": "Price", "type": "integer"}, "description": {"description": "description", "title": "Description", "type": "string"}, "reason": {"description": "Why recommend", "title": "Reason", "type": "string"}}, "required": ["flower_type", "price", "description", "reason"]}
```

